# Always change file name to begin this program
👆👆👆👆👆👆👆👆👆👆👆👆👆👆👆👆👆👆👆

In [1]:
# import library
import pandas as pd
import numpy as np
import geopandas as gpd
from geopy.geocoders import Nominatim

In [2]:
data = pd.read_excel('data2/all_states_all_years_v2.xlsx')
data.head()

,Year,Date_Temp,Date,State,Region,Place,Name,Line Check
0,2015.0,1/1,1/1/2015,Johor,Tangkak,Kg. Seberang Gajah,"Kg. Seberang Gajah, Tangkak",1
1,2015.0,1/1,1/1/2015,Johor,Tangkak,Kg. Sialang Dalam,"Kg. Sialang Dalam, Tangkak",1
2,2015.0,3/1,3/1/2015,Johor,Kota Tinggi,Kg. Lepau,"Kg. Lepau, Kota Tinggi",1
3,2015.0,12/8,12/8/2015,Johor,Tangkak,Kg. Seri Makmor,"Kg. Seri Makmor, Tangkak",1
4,2015.0,12/8,12/8/2015,Johor,Tangkak,Kg. Seberang Gajah,"Kg. Seberang Gajah, Tangkak",1


In [3]:
# testing geolocator library
geolocator = Nominatim(user_agent="u2004763@siswa.um.edu.my")
location = geolocator.geocode("Johor Bahru") # testing

print(location.point)
print(location.address)

# print out longitude and latitude
point = location.point
print("Latitude:", point.latitude)
print("Longitude:", point.longitude)

1 30m 12.798s N, 103 44m 58.411s E
Johor Bahru, Iskandar Malaysia, Johor, 80350, Malaysia
Latitude: 1.503555
Longitude: 103.7495586


In [4]:
# geocoding function
def my_geocoder(row):
    try:
        point = geolocator.geocode(row).point
        return pd.Series({'Latitude': point.latitude, 'Longitude': point.longitude})
    except:
        return pd.Series({'Latitude': None, 'Longitude': None})

data[['Latitude', 'Longitude']] = data.apply(lambda x: my_geocoder(x['Name']), axis=1)

# check the percentage of data successfully geocoded
print("{}% of addresses were geocoded!".format(
    (1 - sum(np.isnan(data["Latitude"])) / len(data)) * 100))

31.74317257408483% of addresses were geocoded!


In [5]:
# export file including NaN for checking
data.to_csv('data2/all_states_all_years_geocoded_nan_v2.csv', index=False)

In [6]:
# Drop location that were not successfully geocoded
data = data.loc[~np.isnan(data["Latitude"])]
data = gpd.GeoDataFrame(
    data, geometry=gpd.points_from_xy(data.Longitude, data.Latitude))
data.crs = {'init': 'epsg:4326'}
data.head()

c:\Users\Khor Kean Teng\AppData\Local\Programs\Python\Python311\Lib\site-packages\pyproj\crs\crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)


,Year,Date_Temp,Date,State,Region,Place,Name,Line Check,Latitude,Longitude,geometry
2,2015.0,3/1,3/1/2015,Johor,Kota Tinggi,Kg. Lepau,"Kg. Lepau, Kota Tinggi",1,1.389603,104.149358,POINT (104.14936 1.38960)
6,2015.0,22/9,22/9/2015,Johor,Johor Bahru,Kg. Pasir,"Kg. Pasir, Johor Bahru",1,1.496672,103.701202,POINT (103.70120 1.49667)
10,2015.0,5/11,5/11/2015,Johor,Johor Bahru,Kg. Pasir,"Kg. Pasir, Johor Bahru",1,1.496672,103.701202,POINT (103.70120 1.49667)
11,2015.0,11/11,11/11/2015,Johor,Tangkak,"Kg. Pulau Penarik, Sg. Mati","Kg. Pulau Penarik, Sg. Mati, Tangkak",1,2.137150,102.567195,POINT (102.56720 2.13715)
14,2015.0,11/11,11/11/2015,Johor,Kluang,Kg. Paya Mas,"Kg. Paya Mas, Kluang",1,1.844126,103.301476,POINT (103.30148 1.84413)


In [7]:
# export csv
data.to_csv('data2/all_states_all_years_geocoded_v2.csv', index=False)